In [1]:
from models.generator import SEMamba
import torch
from utils.util import load_config
device = torch.device('cpu')

In [2]:
global device
config = '/disk4/chocho/SEMamba/recipes/SEMamba_advanced_PCS/SEMamba_advanced_PCS.yaml'
cfg = load_config(config)

In [3]:
cfg['model_cfg']['depth'] = 3
cfg['model_cfg']

{'depth': 3,
 'hid_feature': 32,
 'compress_factor': 0.3,
 'num_tfmamba': 2,
 'd_state': 8,
 'd_conv': 3,
 'expand': 2,
 'norm_epsilon': 1e-05,
 'beta': 2.0,
 'input_channel': 2,
 'output_channel': 1}

In [4]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

In [38]:
import pandas as pd
from itertools import product
import torch
import os

# 定義儲存資料夾
output_dir = "_202505/cal_model_size/"
os.makedirs(output_dir, exist_ok=True)  # 創建資料夾（如果不存在）

# 定義參數範圍
depth_values = [1, 2, 3, 4, 5, 6]
h_values = [2**i for i in range(3, 7)]  # [8, 16, 32, 64]
tf_values = [1, 2, 3, 4]
ds_values = [2**i for i in range(2, 6)]  # [4, 8, 16, 32]
dc_values = [1, 2, 3]
ex_values = [2, 3, 4]

# 生成所有參數組合
input_data = list(product(depth_values, h_values, tf_values, ds_values, dc_values, ex_values))

# 初始化 DataFrame 的資料
data = []
for idx, (depth, h, tf, ds, dc, ex) in enumerate(input_data):
    cfg['model_cfg']['depth'] = depth
    cfg['model_cfg']['hid_feature'] = h
    cfg['model_cfg']['num_tfmamba'] = tf
    cfg['model_cfg']['d_state'] = ds
    cfg['model_cfg']['d_conv'] = dc
    cfg['model_cfg']['expand'] = ex
    model = SEMamba(cfg).to(device)
    param_count = count_parameters(model)
    
    # 儲存模型參數到 .pth 檔案，指定資料夾
    pth_filename = os.path.join(output_dir, f"d{depth}_h{h}_tf{tf}_ds{ds}_dc{dc}_ex{ex}.pth")
    torch.save(model.state_dict(), pth_filename)
    
    # 計算 .pth 檔案大小 (單位: KB)
    file_size_kb = os.path.getsize(pth_filename) / 1024  # 轉換為 KB
    
    # 將每一組參數、參數量及檔案大小添加到資料列表
    data.append({
        'depth': depth,
        'hid_feature': h,
        'num_tfmamba': tf,
        'd_state': ds,
        'd_conv': dc,
        'expand': ex,
        '#parameters': param_count,
        'model_size': file_size_kb
    })
    
    # 列印當前參數組合、參數量及檔案大小
    print(f"depth: {depth}, hid_feature: {h}, num_tfmamba: {tf}, d_state: {ds}, d_conv: {dc}, expand: {ex}")
    print(f"模型的總參數個數: {param_count}")
    print(f"模型檔案大小: {file_size_kb:.2f} KB")
    print(f"已儲存模型至: {pth_filename}")

# 將資料轉換為 DataFrame
df = pd.DataFrame(data)

# 將 DataFrame 儲存為 .pkl 檔案，指定資料夾
pkl_filename = os.path.join(output_dir, "_model_parameters.pkl")
df.to_pickle(pkl_filename)

# 顯示 DataFrame 的前幾行
print("\nDataFrame 前幾行：")
print(df.head())
print(f"\nDataFrame 已儲存為 {pkl_filename}")

depth: 1, hid_feature: 8, num_tfmamba: 1, d_state: 4, d_conv: 1, expand: 2
模型的總參數個數: 5745
模型檔案大小: 55.33 KB
已儲存模型至: _202505/cal_model_size/d1_h8_tf1_ds4_dc1_ex2.pth
depth: 1, hid_feature: 8, num_tfmamba: 1, d_state: 4, d_conv: 1, expand: 3
模型的總參數個數: 7089
模型檔案大小: 59.83 KB
已儲存模型至: _202505/cal_model_size/d1_h8_tf1_ds4_dc1_ex3.pth
depth: 1, hid_feature: 8, num_tfmamba: 1, d_state: 4, d_conv: 1, expand: 4
模型的總參數個數: 8433
模型檔案大小: 65.83 KB
已儲存模型至: _202505/cal_model_size/d1_h8_tf1_ds4_dc1_ex4.pth
depth: 1, hid_feature: 8, num_tfmamba: 1, d_state: 4, d_conv: 2, expand: 2
模型的總參數個數: 5809
模型檔案大小: 55.58 KB
已儲存模型至: _202505/cal_model_size/d1_h8_tf1_ds4_dc2_ex2.pth
depth: 1, hid_feature: 8, num_tfmamba: 1, d_state: 4, d_conv: 2, expand: 3
模型的總參數個數: 7185
模型檔案大小: 60.33 KB
已儲存模型至: _202505/cal_model_size/d1_h8_tf1_ds4_dc2_ex3.pth
depth: 1, hid_feature: 8, num_tfmamba: 1, d_state: 4, d_conv: 2, expand: 4
模型的總參數個數: 8561
模型檔案大小: 66.33 KB
已儲存模型至: _202505/cal_model_size/d1_h8_tf1_ds4_dc2_ex4.pth
depth: 1, hid_fe

In [40]:
df[df['model_size']<650]

,depth,hid_feature,num_tfmamba,d_state,d_conv,expand,#parameters,model_size
0,1,8,1,4,1,2,5745,55.333984
1,1,8,1,4,1,3,7089,59.833984
2,1,8,1,4,1,4,8433,65.833984
3,1,8,1,4,2,2,5809,55.583984
4,1,8,1,4,2,3,7185,60.333984
...,...,...,...,...,...,...,...,...
3019,6,8,4,32,2,3,87777,452.290039
3020,6,8,4,32,2,4,104033,518.352539
3021,6,8,4,32,3,2,71777,392.290039
3022,6,8,4,32,3,3,88161,453.290039


In [ ]:
df['ratio'] = df['#parameters']/df['model_size']

In [46]:
df

,depth,hid_feature,num_tfmamba,d_state,d_conv,expand,#parameters,model_size,ratio
0,1,8,1,4,1,2,5745,55.333984,103.824080
1,1,8,1,4,1,3,7089,59.833984,118.477819
2,1,8,1,4,1,4,8433,65.833984,128.094936
3,1,8,1,4,2,2,5809,55.583984,104.508521
4,1,8,1,4,2,3,7185,60.333984,119.087113
...,...,...,...,...,...,...,...,...,...
3451,6,64,4,32,2,3,3357009,13229.144531,253.758585
3452,6,64,4,32,2,4,3665233,14433.394531,253.941163
3453,6,64,4,32,3,2,3050833,12033.144531,253.535806
3454,6,64,4,32,3,3,3360081,13241.144531,253.760617


In [5]:
# 輸入數據
input_data = [
    (2, 2, 2, 2, 2, 2), (2, 2, 2, 4, 4, 4), (2, 2, 2, 6, 6, 6),
    (2, 4, 4, 2, 2, 2), (2, 4, 4, 4, 4, 4), (2, 4, 4, 6, 6, 6),
    (2, 6, 6, 2, 2, 2), (2, 6, 6, 4, 4, 4), (2, 6, 6, 6, 6, 6),
    (4, 2, 2, 2, 2, 2), (4, 2, 2, 4, 4, 4), (4, 2, 2, 6, 6, 6),
    (4, 4, 4, 2, 2, 2), (4, 4, 4, 4, 4, 4), (4, 4, 4, 6, 6, 6),
    (4, 6, 6, 2, 2, 2), (4, 6, 6, 4, 4, 4), (4, 6, 6, 6, 6, 6),
    (6, 2, 2, 2, 2, 2), (6, 2, 2, 4, 4, 4), (6, 2, 2, 6, 6, 6),
    (6, 4, 4, 2, 2, 2), (6, 4, 4, 4, 4, 4), (6, 4, 4, 6, 6, 6),
    (6, 6, 6, 2, 2, 2), (6, 6, 6, 4, 4, 4), (6, 6, 6, 6, 6, 6),
]

In [31]:
[2**i for i in range(2, 5)]

[4, 8, 16]

In [ ]:
from itertools import product

# 定義參數範圍
depth_values = [1, 2, 3, 4, 5, 6]
h_values = [2**i for i in range(3, 7)]  # [2, 4, 8, 16, 32, 64]
tf_values = [1, 2, 3, 4]
ds_values = [2**i for i in range(2, 6)]  # [2, 4, 8, 16, 32, 64]
dc_values = [1, 2, 3]
ex_values = [2, 3, 4]
input_data = list(product(depth_values, h_values, tf_values, ds_values, dc_values, ex_values))

y_data = []
for depth, h, tf, ds, dc, ex in input_data:
    cfg['model_cfg']['depth'] = depth
    cfg['model_cfg']['hid_feature'] = h
    cfg['model_cfg']['num_tfmamba'] = tf
    cfg['model_cfg']['d_state'] = ds
    cfg['model_cfg']['d_conv'] = dc
    cfg['model_cfg']['expand'] = ex
    model = SEMamba(cfg).to(device)
    print()
    print("depth, h, tf, ds, dc, ex:", depth, h, tf, ds, dc, ex)
    print(f"模型的總參數個數: {count_parameters(model)}")
    y_data.append(count_parameters(model))


depth, h, tf, ds, dc, ex: 1 2 1 2 1 2
模型的總參數個數: 729

depth, h, tf, ds, dc, ex: 1 2 1 2 1 4
模型的總參數個數: 1017

depth, h, tf, ds, dc, ex: 1 2 1 2 2 2
模型的總參數個數: 745

depth, h, tf, ds, dc, ex: 1 2 1 2 2 4
模型的總參數個數: 1049

depth, h, tf, ds, dc, ex: 1 2 1 2 3 2
模型的總參數個數: 761

depth, h, tf, ds, dc, ex: 1 2 1 2 3 4
模型的總參數個數: 1081

depth, h, tf, ds, dc, ex: 1 2 1 4 1 2
模型的總參數個數: 825

depth, h, tf, ds, dc, ex: 1 2 1 4 1 4
模型的總參數個數: 1209

depth, h, tf, ds, dc, ex: 1 2 1 4 2 2
模型的總參數個數: 841

depth, h, tf, ds, dc, ex: 1 2 1 4 2 4
模型的總參數個數: 1241

depth, h, tf, ds, dc, ex: 1 2 1 4 3 2
模型的總參數個數: 857

depth, h, tf, ds, dc, ex: 1 2 1 4 3 4
模型的總參數個數: 1273

depth, h, tf, ds, dc, ex: 1 2 1 8 1 2
模型的總參數個數: 1017

depth, h, tf, ds, dc, ex: 1 2 1 8 1 4
模型的總參數個數: 1593

depth, h, tf, ds, dc, ex: 1 2 1 8 2 2
模型的總參數個數: 1033

depth, h, tf, ds, dc, ex: 1 2 1 8 2 4
模型的總參數個數: 1625

depth, h, tf, ds, dc, ex: 1 2 1 8 3 2
模型的總參數個數: 1049

depth, h, tf, ds, dc, ex: 1 2 1 8 3 4
模型的總參數個數: 1657

depth, h, tf, ds, dc, ex: 1 2 1 1

In [13]:
import numpy as np
np.array(y_data)

array([    729,    1017,     745, ..., 4905041, 3688529, 4911185])

In [14]:
len(y_data)

7776

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.pipeline import make_pipeline

# 轉為 numpy 陣列
X_data = np.array(input_data)

# 假設的 N 值（請替換為實際的 count_parameters(model) 結果）
y = np.array([
    y_data
])

# 多項式回歸（degree=2）
polyreg = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
polyreg.fit(X_data, y.reshape(-1, 1))

# Lasso 回歸（用於特徵選擇）
lasso = make_pipeline(PolynomialFeatures(degree=2), Lasso(alpha=0.1, max_iter=10000))
lasso.fit(X_data, y.reshape(-1, 1))

# 獲取多項式特徵名稱和係數
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X_data)
feature_names = poly.get_feature_names_out(['depth', 'h', 'tf', 'ds', 'dc', 'ex'])

# 多項式回歸的係數
lr_model = polyreg.named_steps['linearregression']
poly_coefficients = lr_model.coef_.flatten()

# Lasso 回歸的係數
lasso_model = lasso.named_steps['lasso']
lasso_coefficients = lasso_model.coef_.flatten()

# 輸出多項式（只顯示非零係數）
print("=== 多項式回歸結果 ===")
poly_formula = []
for coef, name in zip(poly_coefficients, feature_names):
    if abs(coef) > 1e-5:  # 忽略接近 0 的係數
        poly_formula.append(f"{coef:.2f} * {name}")
if poly_formula:
    print("$$ N \\approx", " + ".join(poly_formula), "$$")
else:
    print("無顯著項")

print("\n=== Lasso 回歸結果 ===")
lasso_formula = []
for coef, name in zip(lasso_coefficients, feature_names):
    if abs(coef) > 1e-5:  # 忽略接近 0 的係數
        lasso_formula.append(f"{coef:.2f} * {name}")
if lasso_formula:
    print("$$ N \\approx", " + ".join(lasso_formula), "$$")
else:
    print("無顯著項")

# 計算預測誤差
y_pred_poly = polyreg.predict(X_data)
y_pred_lasso = lasso.predict(X_data)
print("\n=== 擬合誤差 ===")
print(f"多項式回歸均方誤差: {np.mean((y_pred_poly - y_data) ** 2):.2f}")
print(f"Lasso 回歸均方誤差: {np.mean((y_pred_lasso - y_data) ** 2):.2f}")

/disk4/chocho/speechbrain/.speechbrain/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.856e+13, tolerance: 4.569e+11
  model = cd_fast.enet_coordinate_descent(


=== 多項式回歸結果 ===
$$ N \approx -136764.67 * depth + -48586.02 * h + -87125.31 * tf + -5292.00 * ds + -1764.00 * dc + -2687.90 * ex + 12285.00 * depth^2 + 7109.60 * depth h + 406.39 * h^2 + 3548.16 * h tf + 126.00 * h ds + 42.00 * h dc + 3825.83 * h ex + 756.00 * tf ds + 252.00 * tf dc + 17098.67 * tf ex + 882.00 * ds ex + 294.00 * dc ex + -16127.42 * ex^2 $$

=== Lasso 回歸結果 ===
$$ N \approx -136762.95 * depth + -48586.00 * h + -87123.56 * tf + -5291.98 * ds + -1755.26 * dc + 58678.98 * ex + 12284.87 * depth^2 + 7109.60 * depth h + -0.07 * depth tf + -0.00 * depth ds + -0.23 * depth dc + 406.39 * h^2 + 3548.16 * h tf + 126.00 * h ds + 41.99 * h dc + 3825.83 * h ex + -0.10 * tf^2 + 756.00 * tf ds + 251.67 * tf dc + 17098.65 * tf ex + -0.00 * ds^2 + -0.01 * ds dc + 882.00 * ds ex + -1.35 * dc^2 + 293.70 * dc ex + -26355.15 * ex^2 $$

=== 擬合誤差 ===
多項式回歸均方誤差: 1164344845850.59
Lasso 回歸均方誤差: 10818299287.16


In [23]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from scipy.optimize import nnls

# 生成多項式特徵（degree=2）
poly = PolynomialFeatures(degree=4)
X_poly = poly.fit_transform(X_data)
feature_names = poly.get_feature_names_out(['depth', 'h', 'tf', 'ds', 'dc', 'ex'])

# 方法 1：非負最小二乘法 (NNLS)
nnls_coefficients, _ = nnls(X_poly, y_data)

# 方法 2：非負 Lasso 回歸
lasso = Lasso(alpha=0.1, max_iter=10000, positive=True)  # positive=True 強制係數非負
lasso.fit(X_poly, y_data)
lasso_coefficients = lasso.coef_

# 輸出多項式（只顯示非零係數）
print("=== 非負最小二乘法 (NNLS) 結果 ===")
nnls_formula = []
for coef, name in zip(nnls_coefficients, feature_names):
    if abs(coef) > 1e-5:  # 忽略接近 0 的係數
        nnls_formula.append(f"{coef:.2f} \\cdot {name}")
if nnls_formula:
    print("$$ N \\approx", " + ".join(nnls_formula), "$$")
else:
    print("無顯著項")

print("\n=== 非負 Lasso 回歸結果 ===")
lasso_formula = []
for coef, name in zip(lasso_coefficients, feature_names):
    if abs(coef) > 1e-5:  # 忽略接近 0 的係數
        lasso_formula.append(f"{coef:.2f} \\cdot {name}")
if lasso_formula:
    print("$$ N \\approx", " + ".join(lasso_formula), "$$")
else:
    print("無顯著項")

# 計算預測誤差
y_pred_nnls = X_poly @ nnls_coefficients
y_pred_lasso = lasso.predict(X_poly)
print("\n=== 擬合誤差 ===")
print(f"非負最小二乘法均方誤差: {np.mean((y_pred_nnls - y_data) ** 2):.2f}")
print(f"非負 Lasso 回歸均方誤差: {np.mean((y_pred_lasso - y_data) ** 2):.2f}")

=== 非負最小二乘法 (NNLS) 結果 ===
$$ N \approx 208.80 \cdot 1 + 18.08 \cdot h + 12.00 \cdot depth h + 9.00 \cdot h^2 + 8.39 \cdot h tf + 20.91 \cdot tf ex + 13.50 \cdot depth h^2 + 3.89 \cdot h^2 tf + 10.73 \cdot h tf ex + 13.50 \cdot depth^2 h^2 + 0.00 \cdot h^3 tf + 0.00 \cdot h^3 ex + 12.51 \cdot h^2 tf ex + 0.00 \cdot h tf^2 ex + 12.00 \cdot h tf ds ex + 4.00 \cdot h tf dc ex + 0.00 \cdot tf^2 ex^2 + 0.19 \cdot tf ex^3 $$

=== 非負 Lasso 回歸結果 ===
$$ N \approx 4.29 \cdot depth + 33.04 \cdot h + 3.75 \cdot depth h + 8.64 \cdot h^2 + 23.13 \cdot h tf + 19.11 \cdot tf ex + 1.06 \cdot depth^2 h + 13.64 \cdot depth h^2 + 0.02 \cdot depth h tf + 0.00 \cdot depth h ds + 0.02 \cdot depth h dc + 0.03 \cdot depth h ex + 0.00 \cdot h^3 + 3.88 \cdot h^2 tf + 13.48 \cdot depth^2 h^2 + 0.00 \cdot h^3 tf + 0.00 \cdot h^2 tf^2 + 0.00 \cdot h^2 tf dc + 12.52 \cdot h^2 tf ex + 12.00 \cdot h tf ds ex + 3.99 \cdot h tf dc ex + 1.77 \cdot h tf ex^2 + 0.01 \cdot tf^2 ex^2 + 0.00 \cdot tf ds ex^2 + 0.02 \cdot tf dc

In [22]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from scipy.optimize import nnls

# 生成多項式特徵（degree=2）
poly = PolynomialFeatures(degree=5)
X_poly = poly.fit_transform(X_data)
feature_names = poly.get_feature_names_out(['depth', 'h', 'tf', 'ds', 'dc', 'ex'])

# 方法 1：非負最小二乘法 (NNLS)
nnls_coefficients, _ = nnls(X_poly, y_data)

# 方法 2：非負 Lasso 回歸
lasso = Lasso(alpha=0.1, max_iter=10000, positive=True)  # positive=True 強制係數非負
lasso.fit(X_poly, y_data)
lasso_coefficients = lasso.coef_

# 輸出多項式（只顯示非零係數）
print("=== 非負最小二乘法 (NNLS) 結果 ===")
nnls_formula = []
for coef, name in zip(nnls_coefficients, feature_names):
    if abs(coef) > 1e-5:  # 忽略接近 0 的係數
        nnls_formula.append(f"{coef:.2f} \\cdot {name}")
if nnls_formula:
    print("$$ N \\approx", " + ".join(nnls_formula), "$$")
else:
    print("無顯著項")

print("\n=== 非負 Lasso 回歸結果 ===")
lasso_formula = []
for coef, name in zip(lasso_coefficients, feature_names):
    if abs(coef) > 1e-5:  # 忽略接近 0 的係數
        lasso_formula.append(f"{coef:.2f} \\cdot {name}")
if lasso_formula:
    print("$$ N \\approx", " + ".join(lasso_formula), "$$")
else:
    print("無顯著項")

# 計算預測誤差
y_pred_nnls = X_poly @ nnls_coefficients
y_pred_lasso = lasso.predict(X_poly)
print("\n=== 擬合誤差 ===")
print(f"非負最小二乘法均方誤差: {np.mean((y_pred_nnls - y_data) ** 2):.2f}")
print(f"非負 Lasso 回歸均方誤差: {np.mean((y_pred_lasso - y_data) ** 2):.2f}")

=== 非負最小二乘法 (NNLS) 結果 ===
$$ N \approx 209.00 \cdot 1 + 18.00 \cdot h + 12.00 \cdot depth h + 9.00 \cdot h^2 + 6.00 \cdot h tf + 23.13 \cdot tf ex + 13.50 \cdot depth h^2 + 4.00 \cdot h^2 tf + 11.54 \cdot h tf ex + 13.50 \cdot depth^2 h^2 + 12.48 \cdot h^2 tf ex + 12.00 \cdot h tf ds ex + 4.00 \cdot h tf dc ex + 0.00 \cdot h^3 tf ex $$

=== 非負 Lasso 回歸結果 ===
$$ N \approx 110.94 \cdot h + 42.90 \cdot tf + 0.22 \cdot h^2 + 0.15 \cdot tf ds + 6.13 \cdot tf dc + 14.18 \cdot depth^2 h + 22.93 \cdot depth h^2 + 16.73 \cdot h^2 tf + 0.05 \cdot h^2 ex + 8.80 \cdot h tf ds + 1.57 \cdot h tf dc + 14.83 \cdot h tf ex + 9.78 \cdot depth^2 h^2 + 2.95 \cdot h^2 tf ex + 5.38 \cdot h tf ds ex + 1.07 \cdot h tf dc ex + 0.17 \cdot h tf ex^2 + 0.00 \cdot tf ds^2 ex + 0.01 \cdot tf ds dc ex + 0.02 \cdot tf ds ex^2 + 1.88 \cdot tf dc^2 ex + 0.65 \cdot tf dc ex^2 + 0.28 \cdot depth^3 h^2 + 0.01 \cdot depth^2 h^3 + 0.00 \cdot depth^2 h^2 tf + 0.00 \cdot depth^2 h^2 ds + 0.00 \cdot depth^2 h^2 dc + 0.00 \cdot

In [24]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from scipy.optimize import nnls

# 生成多項式特徵（degree=2）
poly = PolynomialFeatures(degree=6)
X_poly = poly.fit_transform(X_data)
feature_names = poly.get_feature_names_out(['depth', 'h', 'tf', 'ds', 'dc', 'ex'])

# 方法 1：非負最小二乘法 (NNLS)
nnls_coefficients, _ = nnls(X_poly, y_data)

# 方法 2：非負 Lasso 回歸
lasso = Lasso(alpha=0.1, max_iter=10000, positive=True)  # positive=True 強制係數非負
lasso.fit(X_poly, y_data)
lasso_coefficients = lasso.coef_

# 輸出多項式（只顯示非零係數）
print("=== 非負最小二乘法 (NNLS) 結果 ===")
nnls_formula = []
for coef, name in zip(nnls_coefficients, feature_names):
    if abs(coef) > 1e-5:  # 忽略接近 0 的係數
        nnls_formula.append(f"{coef:.2f} \\cdot {name}")
if nnls_formula:
    print("$$ N \\approx", " + ".join(nnls_formula), "$$")
else:
    print("無顯著項")

print("\n=== 非負 Lasso 回歸結果 ===")
lasso_formula = []
for coef, name in zip(lasso_coefficients, feature_names):
    if abs(coef) > 1e-5:  # 忽略接近 0 的係數
        lasso_formula.append(f"{coef:.2f} \\cdot {name}")
if lasso_formula:
    print("$$ N \\approx", " + ".join(lasso_formula), "$$")
else:
    print("無顯著項")

# 計算預測誤差
y_pred_nnls = X_poly @ nnls_coefficients
y_pred_lasso = lasso.predict(X_poly)
print("\n=== 擬合誤差 ===")
print(f"非負最小二乘法均方誤差: {np.mean((y_pred_nnls - y_data) ** 2):.2f}")
print(f"非負 Lasso 回歸均方誤差: {np.mean((y_pred_lasso - y_data) ** 2):.2f}")

=== 非負最小二乘法 (NNLS) 結果 ===
$$ N \approx 209.00 \cdot 1 + 18.00 \cdot h + 12.00 \cdot depth h + 9.00 \cdot h^2 + 6.00 \cdot h tf + 23.13 \cdot tf ex + 13.50 \cdot depth h^2 + 4.00 \cdot h^2 tf + 11.54 \cdot h tf ex + 13.50 \cdot depth^2 h^2 + 12.48 \cdot h^2 tf ex + 12.00 \cdot h tf ds ex + 4.00 \cdot h tf dc ex + 0.00 \cdot h^3 tf ex $$

=== 非負 Lasso 回歸結果 ===
$$ N \approx 139.97 \cdot h + 54.89 \cdot tf + 0.41 \cdot h^2 + 0.15 \cdot tf ds + 3.86 \cdot tf dc + 6.26 \cdot depth^2 h + 18.92 \cdot depth h^2 + 0.03 \cdot h^3 + 23.38 \cdot h^2 tf + 0.00 \cdot h^2 ds + 0.08 \cdot h^2 dc + 0.12 \cdot h^2 ex + 16.59 \cdot h tf ds + 5.07 \cdot h tf dc + 13.15 \cdot h tf ex + 11.53 \cdot depth^2 h^2 + 0.82 \cdot h tf ds ex + 0.02 \cdot tf ds ex^2 + 0.71 \cdot tf dc ex^2 + 0.25 \cdot tf ex^3 + 0.16 \cdot depth^3 h^2 + 0.00 \cdot depth^2 h^3 + 0.00 \cdot depth^2 h^2 tf + 0.00 \cdot depth^2 h^2 ds + 0.00 \cdot depth^2 h^2 dc + 0.00 \cdot depth^2 h^2 ex + 0.00 \cdot h^2 tf ds ex + 0.95 \cdot h^2 tf ex

In [12]:
import torch
from torchinfo import summary

batch, length, dim = 1, 201, 286
print("B,L,D:",batch, length, dim)
x = torch.randn(batch, length, dim).to(device)
summary_str = summary(model, input_size=[x.shape, x.shape], depth=15, col_names=("input_size", "output_size", "num_params"), verbose=0)
print(summary_str)

B,L,D: 1 201 286
Layer (type:depth-idx)                             Input Shape               Output Shape              Param #
SEMamba                                            [1, 201, 286]             [1, 201, 286]             --
├─DenseEncoder: 1-1                                [1, 2, 286, 201]          [1, 2, 286, 100]          --
│    └─Sequential: 2-1                             [1, 2, 286, 201]          [1, 2, 286, 201]          --
│    │    └─Conv2d: 3-1                            [1, 2, 286, 201]          [1, 2, 286, 201]          6
│    │    └─InstanceNorm2d: 3-2                    [1, 2, 286, 201]          [1, 2, 286, 201]          4
│    │    └─PReLU: 3-3                             [1, 2, 286, 201]          [1, 2, 286, 201]          2
│    └─DenseBlock: 2-2                             [1, 2, 286, 201]          [1, 2, 286, 201]          --
│    │    └─ModuleList: 3-4                        --                        --                        --
│    │    │    └─Sequential

In [34]:
pwd

'/disk4/chocho/SEMamba'